In [1]:
import pandas as pd
import numpy as np
import datetime

### Fankhauser
- supply policy
- 188 countries

In [2]:
documents = pd.read_excel("../../data/climate_policy/CPR Master for Sharing.xlsx",sheet_name="Documents")
events = pd.read_excel("../../data/climate_policy/CPR Master for Sharing.xlsx",sheet_name="Events")

In [3]:
documents_mitigation = documents.copy()
#documents_mitigation["Responses"] = documents_mitigation["Responses"].fillna("None")
#documents_mitigation[documents_mitigation["Responses"].str.contains('Mitigation')]

documents_mitigation["Keywords"] = documents_mitigation["Keywords"].fillna("None")
documents_mitigation = documents_mitigation[documents_mitigation["Keywords"].str.contains('Energy Supply')] #'air pollution'
documents_mitigation = documents_mitigation.query("`Document role`=='MAIN'")

In [5]:
unique_mitigation = documents_mitigation["CPR Family ID"].unique()
this_policy = unique_mitigation[0]
for this_policy in unique_mitigation:
    pass_event = events.query("`Event type` == 'Passed/Approved'").query("`CPR Family ID`==@this_policy")
    if len(pass_event)>0:
        pass_event_year = int(str(pass_event["Date"].iloc[0])[:4])
        this_policy_index = documents_mitigation.query("`CPR Family ID`==@this_policy").index
        documents_mitigation.loc[this_policy_index,"Passed_Year"] = pass_event_year
no_match = documents_mitigation[documents_mitigation["Passed_Year"].isna()==True]["CPR Family ID"] # documents without events that match them
documents_mitigation = documents_mitigation[documents_mitigation["Passed_Year"].isna()==False]
documents_mitigation

,ID,Document ID,Collection name,Collection summary,Document title,Family name,Family summary,Document role,Document variant,Geography ISO,...,Language,Keywords,Geography,CPR Document ID,CPR Family ID,CPR Collection ID,CPR Family Slug,CPR Document Slug,CPR Document Status,Passed_Year
0,1001,0,NaN,NaN,Regulatory Order fixing the guaranteed purchas...,Regulatory Order fixing the guaranteed purchas...,<p>Establishes a variable 20-year Solar PV fee...,MAIN,NaN,DZA,...,NaN,Energy Supply,Algeria,CCLW.executive.1001.0,CCLW.family.1001.0,NaN,regulatory-order-fixing-the-guaranteed-purchas...,regulatory-order-fixing-the-guaranteed-purchas...,PUBLISHED,2014.0
1,1002,0,NaN,NaN,The National Climate Plan,The National Climate Plan,The National Climate Plan presents an agenda b...,MAIN,NaN,DZA,...,NaN,Adaptation;Energy Supply;Energy Demand;Redd+ A...,Algeria,CCLW.executive.1002.0,CCLW.family.1002.0,NaN,the-national-climate-plan_01b8,the-national-climate-plan_2c08,PUBLISHED,2013.0
2,1003,0,NaN,NaN,Executive Decree No. 2011-423 fixing the opera...,Executive Decree No. 2011-423 fixing the opera...,Amends the operating procedures of the Nationa...,MAIN,NaN,DZA,...,NaN,Research And Development;Energy Supply,Algeria,CCLW.executive.1003.0,CCLW.family.1003.0,NaN,executive-decree-no-2011-423-fixing-the-operat...,executive-decree-no-2011-423-fixing-the-operat...,PUBLISHED,2011.0
3,1004,2230,NaN,NaN,Renewable Energy and Energy Efficiency Develop...,Renewable Energy and Energy Efficiency Develop...,The main objective of the Renewable Energy and...,MAIN,NaN,DZA,...,NaN,Energy Supply;Energy Demand,Algeria,CCLW.executive.1004.2230,CCLW.family.1004.0,NaN,renewable-energy-and-energy-efficiency-develop...,renewable-energy-and-energy-efficiency-develop...,PUBLISHED,2011.0
6,1007,0,NaN,NaN,Executive Decree No. 2006-138 regulating the e...,Executive Decree No. 2006-138 regulating the e...,Establishes emission limits for stationary sou...,MAIN,NaN,DZA,...,NaN,Energy Supply,Algeria,CCLW.executive.1007.0,CCLW.family.1007.0,NaN,executive-decree-no-2006-138-regulating-the-em...,executive-decree-no-2006-138-regulating-the-em...,PUBLISHED,2006.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4215,9368,rtl_154,NaN,NaN,Energy Agreement for Sustainable Growth,Energy Accord for Sustainable Growth (Energiea...,The Energy Agreement for sustainable growth&nb...,MAIN,NaN,NLD,...,NaN,Energy Supply;Energy Demand,NaN,CCLW.executive.9368.rtl_154,CCLW.family.9368.0,NaN,energy-accord-for-sustainable-growth-energieak...,information-on-the-energy-agreement-for-sustai...,PUBLISHED,2013.0
4225,9454,rtl_164,NaN,NaN,Concept note for ensuring electricity supply i...,Concept note for ensuring electricity supply i...,This document aims at ensuring that Uzbekistan...,MAIN,NaN,UZB,...,NaN,Energy Supply;Energy Demand;Renewables;Electri...,Uzbekistan,CCLW.executive.9454.rtl_164,CCLW.family.9454.0,NaN,concept-note-for-ensuring-electricity-supply-i...,concept-note-for-ensuring-electricity-supply-i...,PUBLISHED,2019.0
4235,9724,rtl_174,NaN,NaN,National Climate Change Adaptation Plan 2021-2030,National Climate Change Adaptation Plan 2021-2030,The National Climate Change Adaptation Plan is...,MAIN,NaN,ESP,...,NaN,Adaptation;Institutions / Administrative Arran...,Spain,CCLW.executive.9724.rtl_174,CCLW.family.9724.0,NaN,national-climate-change-adaptation-plan-2021-2...,national-climate-change-adaptation-plan-2021-2...,PUBLISHED,2020.0
4316,1317,rtl_255,NaN,NaN,National Electricity Plan Volume I (2022-2032),National Electricity Plan,<p>As per Section 3 of the Electricity Act 200...,MAIN,Original Language,IND,...,English,Energy Supply,NaN,CCLW.executive.1317.rtl_255,CCLW.family.1317.0,NaN,national-electricity-plan-generation_5260,national-electricity-plan-volume-i-2022-2032-3a59,PUBLISHED,2012.0


In [21]:
unique_countries = documents_mitigation["Geography ISO"].unique()
min_year = documents_mitigation["Passed_Year"].min()
max_year = documents_mitigation["Passed_Year"].max()
list_year = np.arange(max_year-min_year+1)+min_year

for this_country in unique_countries:
    this_country_doc = documents_mitigation.query("`Geography ISO`==@this_country")
    this_country_count = pd.DataFrame(data={"Year":list_year,"ISO":this_country})
    this_country_count = this_country_count.set_index("Year")
    for yr in list_year:
        docs_year = this_country_doc[this_country_doc["Passed_Year"]<=yr]
        this_country_count.loc[yr,"num_supply_policy"] = len(docs_year)
        
    if this_country == unique_countries[0]:
        all_country_count = this_country_count
    else:
        all_country_count = pd.concat([all_country_count,this_country_count])
all_country_count = all_country_count.reset_index()
all_country_count

,Year,ISO,num_supply_policy
0,1980.0,DZA,0.0
1,1981.0,DZA,0.0
2,1982.0,DZA,0.0
3,1983.0,DZA,0.0
4,1984.0,DZA,0.0
...,...,...,...
8267,2019.0,ALB,3.0
8268,2020.0,ALB,3.0
8269,2021.0,ALB,3.0
8270,2022.0,ALB,3.0


### air quality policy binary

In [64]:
crosswalk = pd.read_excel("../../data/country_names.xlsx",sheet_name="crosswalk")

In [48]:
air_quality = pd.read_excel("../../data/climate_policy/air_quality_pm25.xlsx")
air_quality_data = pd.merge(air_quality[["Column1","Column5"]],crosswalk[["Country Code","AirQual_Name"]],left_on="Column1",right_on="AirQual_Name")
air_quality_data = air_quality_data[air_quality_data["Column1"].isna()==False]
air_quality_data = air_quality_data.rename(columns={"Column5":"start_year"})
air_quality_data = air_quality_data.replace("!",np.nan).replace("<2006","2006").replace("<2009","2009").replace("2010/2014","2010")
air_quality_data = air_quality_data[air_quality_data["start_year"].isna()==False]
air_quality_data

,Column1,start_year,Country Code,AirQual_Name
75,Austria,2015,AUT,Austria
77,Bulgaria,2011,BGR,Bulgaria
78,Croatia,2010,HRV,Croatia
81,Denmark,2010,DNK,Denmark
83,Finland,2011,FIN,Finland
...,...,...,...,...
258,Singapore,2014,SGP,Singapore
260,Tonga,2015,TON,Tonga
263,Viet Nam,2013,VNM,Viet Nam
264,Gibraltar,2010,GIB,Gibraltar


In [51]:
all_country_count_aq = all_country_count.copy()
iso_countries = air_quality_data["Country Code"].unique()
for cntry in iso_countries:
    if len(air_quality_data.query("`Country Code`==@cntry"))!=0:
        start_year_text = air_quality_data.query("`Country Code`==@cntry")["start_year"].iloc[0]
        start_year = float(start_year_text)
        cntry_data = all_country_count_aq.query("ISO==@cntry")
        cntry_data_after_index = cntry_data[cntry_data["Year"]>=start_year].index
        all_country_count_aq.loc[cntry_data_after_index,"binary_airqual"] = 1
all_country_count_aq["binary_airqual"] = all_country_count_aq["binary_airqual"].fillna(0)
all_country_count_aq

,Year,ISO,num_supply_policy,binary_airqual
0,1980.0,DZA,0.0,0.0
1,1981.0,DZA,0.0,0.0
2,1982.0,DZA,0.0,0.0
3,1983.0,DZA,0.0,0.0
4,1984.0,DZA,0.0,0.0
...,...,...,...,...
8267,2019.0,ALB,3.0,1.0
8268,2020.0,ALB,3.0,1.0
8269,2021.0,ALB,3.0,1.0
8270,2022.0,ALB,3.0,1.0


### NDC ambition

In [53]:
ndc_ambition = pd.read_csv("../../data/climate_policy/main_data.csv")
ndc_ambition

,Unnamed: 0,iso,ghg_ambition_n,new_pm_n,change_cond,cw_2030,year1,year2,gdp_pc,polyarchy,...,pre_affect,post_affect,change_affect,pre_cfinance,post_cfinance,change_cfinance,paris_support,protect_env,region,log_gdp_pc
0,1,ALB,1.0,1.0,0.0,1,2016,2021,3952.801215,0.508,...,0.013075,0.001265,0.160896,0.000736,0.001339,0.000603,NaN,NaN,1,8.282180
1,2,AND,0.0,1.0,0.0,0,2017,2020,35770.776704,NaN,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,NaN,0.726096,5,10.484887
2,3,AGO,0.0,1.0,0.0,1,2020,2021,4166.979833,0.353,...,0.001478,0.000000,0.403510,0.000042,0.000000,-0.000042,NaN,NaN,4,8.334947
3,4,ATG,1.0,1.0,0.0,0,2016,2021,14285.329777,NaN,...,0.043356,0.009431,0.235179,0.000314,0.000416,0.000102,NaN,NaN,10,9.566988
4,5,ARG,1.0,1.0,1.0,1,2016,2021,13789.060425,0.762,...,0.001010,0.000355,0.674131,0.000020,0.000109,0.000089,0.807000,0.417747,2,9.531631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,118,USA,1.0,0.0,0.0,1,2016,2021,56863.371496,0.891,...,0.006508,0.000824,0.242950,0.000000,0.000000,0.000000,0.652044,0.525809,5,10.948407
118,119,UZB,1.0,0.0,0.0,1,2018,2021,2615.025134,0.189,...,0.000014,0.000512,1.490226,0.000417,0.001160,0.000743,NaN,NaN,1,7.869029
119,120,VUT,1.0,1.0,0.0,1,2016,2021,2695.666186,0.753,...,0.058437,0.070483,1.202672,0.001634,0.002394,0.000761,NaN,NaN,9,7.899401
120,121,VEN,0.0,1.0,0.0,0,2018,2021,17300.000000,0.217,...,0.001038,0.000033,0.507087,0.000000,0.000000,0.000000,0.853000,NaN,2,9.758462


In [79]:
all_country_count_aq_ndc = pd.merge(all_country_count_aq,ndc_ambition[["iso","ghg_ambition_n","cw_2030"]],
                                    left_on="ISO",right_on="iso",how="left")
all_country_count_aq_ndc = all_country_count_aq_ndc.drop(columns="iso")
all_country_count_aq_ndc

,Year,ISO,num_supply_policy,binary_airqual,ghg_ambition_n,cw_2030
0,1980.0,DZA,0.0,0.0,NaN,NaN
1,1981.0,DZA,0.0,0.0,NaN,NaN
2,1982.0,DZA,0.0,0.0,NaN,NaN
3,1983.0,DZA,0.0,0.0,NaN,NaN
4,1984.0,DZA,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...
8267,2019.0,ALB,3.0,1.0,1.0,1.0
8268,2020.0,ALB,3.0,1.0,1.0,1.0
8269,2021.0,ALB,3.0,1.0,1.0,1.0
8270,2022.0,ALB,3.0,1.0,1.0,1.0


### UNFCCC Annex
https://unfccc.int/sites/default/files/convention_text_with_annexes_english_for_posting.pdf [accessed 18/01/2024]

In [61]:
annex_one_orig = ["Australia","Austria","Belarus","Belgium","Bulgaria","Canada","Denmark","Estonia","Finland",
                  "France","Germany","Greece","Hungary","Iceland","Ireland","Italy","Japan","Latvia","Lithuania","Luxembourg",
                 "Netherlands","New Zealand","Norway","Poland","Portugal","Romania","Russian Federation","Spain","Sweden",
                "Switzerland","Turkey","Ukraine","United Kingdom of Great Britain and Northern Ireland","United States of America"]
annex_one_1998 = ["Croatia","Czech Republic","Liechtenstein","Monaco","Slovakia","Slovenia"]
annex_one_2010 = ["Malta"]
annex_one_2013 = ["Cyprus"]
annex_two = ["Australia","Austria","Belgium","Canada","Denmark","Finland","France","Germany",
             "Greece","Iceland","Ireland","Italy","Japan","Luxembourg","Netherlands","New Zealand","Norway","Portugal","Spain",
             "Sweden","Switzerland","United Kingdom of Great Britain and Northern Ireland","United States of America"]
annex_two_2002exit = ["Turkey"]

In [86]:
all_countries = annex_one_orig + annex_one_1998 + annex_one_2010 + annex_one_2013
year_list = np.arange(32)+1992
for cntry in all_countries:
    cntry_annex_data = pd.DataFrame(data={"year":year_list,"country":cntry})
    cntry_annex_data = cntry_annex_data.set_index("year")
    
    if cntry in annex_one_orig:
        cntry_annex_data.loc[:,"annex_one"] = 1
    elif cntry in annex_one_1998:
        cntry_annex_data.loc[1998:,"annex_one"] = 1
    elif cntry in annex_one_2010:
        cntry_annex_data.loc[2010:,"annex_one"] = 1
    elif cntry in annex_one_2013:
        cntry_annex_data.loc[2013:,"annex_one"] = 1
        
    if cntry in annex_two:
        cntry_annex_data.loc[:,"annex_two"] = 1
    elif cntry in annex_two_2002exit:
        cntry_annex_data.loc[:2002,"annex_two"] = 1
        
    if cntry == all_countries[0]:
        all_annex_data = cntry_annex_data
    else:
        all_annex_data = pd.concat([all_annex_data,cntry_annex_data])
all_annex_data = all_annex_data.reset_index()
all_annex_data = pd.merge(all_annex_data,crosswalk[["Country Code","Annex_Name"]],left_on = "country",right_on="Annex_Name",how="left")
all_annex_data = all_annex_data.drop(columns=["country","Annex_Name"])
all_annex_data

,year,annex_one,annex_two,Country Code
0,1992,1.0,1.0,AUS
1,1993,1.0,1.0,AUS
2,1994,1.0,1.0,AUS
3,1995,1.0,1.0,AUS
4,1996,1.0,1.0,AUS
...,...,...,...,...
1339,2019,1.0,NaN,CYP
1340,2020,1.0,NaN,CYP
1341,2021,1.0,NaN,CYP
1342,2022,1.0,NaN,CYP


In [87]:
all_country_ = pd.merge(all_country_count_aq_ndc,all_annex_data,left_on = ["Year","ISO"],right_on=["year","Country Code"],how="left")
all_country_ = all_country_.drop(columns=["year","Country Code"])
all_country_ = all_country_.fillna(0)
all_country_.set_index("Year").to_csv("_all_temporal_policy.csv")
all_country_

,Year,ISO,num_supply_policy,binary_airqual,ghg_ambition_n,cw_2030,annex_one,annex_two
0,1980.0,DZA,0.0,0.0,0.0,0.0,0.0,0.0
1,1981.0,DZA,0.0,0.0,0.0,0.0,0.0,0.0
2,1982.0,DZA,0.0,0.0,0.0,0.0,0.0,0.0
3,1983.0,DZA,0.0,0.0,0.0,0.0,0.0,0.0
4,1984.0,DZA,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
8267,2019.0,ALB,3.0,1.0,1.0,1.0,0.0,0.0
8268,2020.0,ALB,3.0,1.0,1.0,1.0,0.0,0.0
8269,2021.0,ALB,3.0,1.0,1.0,1.0,0.0,0.0
8270,2022.0,ALB,3.0,1.0,1.0,1.0,0.0,0.0
